# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [62]:
!pip install yfinance



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
# YOUR CHANGES HERE
import yfinance as yf
import pandas as pd

#AAPL is Apple, SPE is SPDER S&P 500 ETF Trust, TSLA is Tesla
#SIMO is Silicon Motion Technology
#SIRI is Sirius XM Holdings
#AU is AngloGold Ashanti Ltd, mining company
tickers = ["AAPL", "SPY", "TSLA", "SIMO", "SIRI", "AU"] 

data = yf.download(tickers, period="24mo", interval="1d", auto_adjust=False)

adj_close = data['Adj Close']

# Get last trading day of each month
monthly_last = adj_close.resample('M').last()

# Drop Sept. 2025
monthly_last = monthly_last.iloc[:-1]  

print("Data covers from:", monthly_last.index[0].date(), "to:", monthly_last.index[-1].date())

print(monthly_last)





[*********************100%***********************]  6 of 6 completed

Data covers from: 2023-09-30 to: 2025-08-31
Ticker            AAPL         AU       SIMO       SIRI         SPY  \
Date                                                                  
2023-09-30  169.549286  14.832229  48.000610  41.959274  417.865662   
2023-10-31  169.113525  16.747276  50.182880  39.731354  408.794373   
2023-11-30  188.355316  18.070911  55.545036  43.682831  446.135223   
2023-12-31  190.913651  17.545212  57.878307  51.056641  466.503693   
2024-01-31  182.851913  16.540751  59.493645  47.509743  473.933441   
2024-02-29  179.461273  17.470112  67.357872  41.476887  498.666534   
2024-03-31  170.257355  21.196465  73.209694  36.409573  514.973877   
2024-04-30  169.115540  21.950750  70.231445  27.588697  494.210358   
2024-05-31  191.138184  23.096504  74.775757  26.696659  519.207397   
2024-06-30  209.401932  23.994013  77.552681  26.791325  537.525085   
2024-07-31  220.795654  26.810659  66.492882  32.660805  544.033997   
2024-08-31  227.939163  28.534407


/tmp/ipykernel_53785/3125041507.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_last = adj_close.resample('M').last()


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [79]:
# YOUR CHANGES HERE

monthly_last 

Ticker,AAPL,AU,SIMO,SIRI,SPY,TSLA
Date,,,,,,
2023-09-30,169.549286,14.832229,48.000610,41.959274,417.865662,250.220001
2023-10-31,169.113525,16.747276,50.182880,39.731354,408.794373,200.839996
2023-11-30,188.355316,18.070911,55.545036,43.682831,446.135223,240.080002
2023-12-31,190.913651,17.545212,57.878307,51.056641,466.503693,248.479996
2024-01-31,182.851913,16.540751,59.493645,47.509743,473.933441,187.289993
2024-02-29,179.461273,17.470112,67.357872,41.476887,498.666534,201.880005
2024-03-31,170.257355,21.196465,73.209694,36.409573,514.973877,175.789993
2024-04-30,169.115540,21.950750,70.231445,27.588697,494.210358,183.279999
2024-05-31,191.138184,23.096504,74.775757,26.696659,519.207397,178.080002


In [69]:
monthly_last.to_csv("historical_prices.tsv", sep='\t', index_label="date")

Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [81]:
# YOUR CHANGES HERE

prices = pd.read_csv("historical_prices.tsv", sep="\t", index_col="date", parse_dates=True)

#monthly returns: (P_t - P_{t-1}) / P_{t-1}
returns = prices.pct_change()

print("Historical Returns:")
print(returns.head())




Historical Returns:
                AAPL        AU      SIMO      SIRI       SPY      TSLA
date                                                                  
2023-09-30       NaN       NaN       NaN       NaN       NaN       NaN
2023-10-31 -0.002570  0.129114  0.045463 -0.053097 -0.021709 -0.197346
2023-11-30  0.113780  0.079036  0.106852  0.099455  0.091344  0.195379
2023-12-31  0.013582 -0.029091  0.042007  0.168803  0.045655  0.034988
2024-01-31 -0.042227 -0.057250  0.027909 -0.069470  0.015926 -0.246257


In [83]:
# Drop the first row (since all NaN)
returns = returns.dropna()

print("Monthly Returns:")
print(returns.head())

Monthly Returns:
                AAPL        AU      SIMO      SIRI       SPY      TSLA
date                                                                  
2023-10-31 -0.002570  0.129114  0.045463 -0.053097 -0.021709 -0.197346
2023-11-30  0.113780  0.079036  0.106852  0.099455  0.091344  0.195379
2023-12-31  0.013582 -0.029091  0.042007  0.168803  0.045655  0.034988
2024-01-31 -0.042227 -0.057250  0.027909 -0.069470  0.015926 -0.246257
2024-02-29 -0.018543  0.056186  0.132186 -0.126981  0.052187  0.077901


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [84]:
# YOUR CHANGES HERE
returns.to_csv("historical_returns.tsv", sep="\t", index_label="date")

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [45]:
# YOUR CHANGES HERE


returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
returns.set_index("date", inplace=True)

estimated_returns = returns.mean()

print(estimated_returns)


AAPL    0.011104
AU      0.060025
SIMO    0.023014
SIRI   -0.015079
SPY     0.016871
TSLA    0.022989
dtype: float64


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [46]:
# YOUR CHANGES HERE

estimated_returns_df = estimated_returns.reset_index()
estimated_returns_df.columns = ["asset", "estimated_return"]

estimated_returns_df.to_csv("estimated_returns.tsv", sep="\t", index=False)



Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [47]:
# YOUR CHANGES HERE

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
returns.set_index("date", inplace=True)

cov_matrix = returns.cov()

print(cov_matrix)


          AAPL        AU      SIMO      SIRI       SPY      TSLA
AAPL  0.003921 -0.001073  0.000685  0.001708  0.001087  0.003355
AU   -0.001073  0.013294 -0.000381  0.002951 -0.000381 -0.004166
SIMO  0.000685 -0.000381  0.009053  0.000045  0.002197  0.000591
SIRI  0.001708  0.002951  0.000045  0.014772  0.001300  0.001074
SPY   0.001087 -0.000381  0.002197  0.001300  0.001424  0.002472
TSLA  0.003355 -0.004166  0.000591  0.001074  0.002472  0.025672


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [48]:
# YOUR CHANGES HERE

#A covariance matrix is a table (matrix) that shows how different variables move together 
      #how asset returns move together. In finance, each variable is the return of an asset, 
      # and the matrix tells us how risky each asset is (variance = diagonal entries)
      # How correlated different assets are (covariance = off-diagonal entries)

cov_matrix.to_csv("estimated_covariance.tsv", sep="\t", index=True)


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [49]:
# YOUR CHANGES HERE

expected_returns = pd.read_csv("estimated_returns.tsv", sep="\t")

max_return_row = expected_returns.loc[expected_returns['estimated_return'].idxmax()]
best_asset = max_return_row['asset']
best_return = max_return_row['estimated_return']

max_return_portfolio = pd.DataFrame({
    'asset': expected_returns['asset'],
    'allocation': [1.0 if asset == best_asset else 0.0 for asset in expected_returns['asset']]
})

print(max_return_portfolio)


  asset  allocation
0  AAPL         0.0
1    AU         1.0
2  SIMO         0.0
3  SIRI         0.0
4   SPY         0.0
5  TSLA         0.0


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [50]:
# YOUR CHANGES HERE

max_return_portfolio.to_csv("maximum_return.tsv", sep="\t", index=False)


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [51]:
# YOUR CHANGES HERE

expected_returns = pd.read_csv("estimated_returns.tsv", sep="\t")

min_return_row = expected_returns.loc[expected_returns['estimated_return'].idxmin()]
worst_asset = min_return_row['asset']
worst_return = min_return_row['estimated_return']

min_return_portfolio = pd.DataFrame({
    'asset': expected_returns['asset'],
    'allocation': [1.0 if asset == worst_asset else 0.0 for asset in expected_returns['asset']]
})

print(min_return_portfolio)


  asset  allocation
0  AAPL         0.0
1    AU         0.0
2  SIMO         0.0
3  SIRI         1.0
4   SPY         0.0
5  TSLA         0.0


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [52]:
# YOUR CHANGES HERE

min_return_portfolio.to_csv("minimum_risk.tsv", sep="\t", index=False)

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [53]:
!pip install cvxpy



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [54]:
# YOUR CHANGES HERE

import cvxpy as cp
import numpy as np

expected_returns = pd.read_csv("estimated_returns.tsv", sep="\t")
cov_matrix = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

mu = expected_returns['estimated_return'].values
assets = expected_returns['asset'].values
Sigma = cov_matrix.loc[assets, assets].values  
n = len(mu)

w = cp.Variable(n)
objective = cp.quad_form(w, Sigma)
constraints = [cp.sum(w) == 1, w >= 0]
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()
w_min_var = w.value
min_risk_return = mu @ w_min_var

max_return = np.max(mu)

target_returns = np.linspace(min_risk_return, max_return, 101)

results = []

for idx, r_target in enumerate(target_returns):
    w = cp.Variable(n)
    objective = cp.quad_form(w, Sigma)
    constraints = [
        cp.sum(w) == 1,
        mu @ w == r_target,
        w >= 0
    ]
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()

    if w.value is not None:
        weights = w.value
        port_return = mu @ weights
        port_variance = weights.T @ Sigma @ weights
        port_risk = np.sqrt(port_variance)
        results.append([idx, port_return, port_risk] + list(weights))
    else:
        print(f"Optimization failed at index {idx}, target return {r_target:.4f}")
        results.append([idx, np.nan, np.nan] + [np.nan]*n)

columns = ['index', 'return', 'risk'] + list(assets)
df = pd.DataFrame(results, columns=columns)





In [55]:
df

,index,return,risk,AAPL,AU,SIMO,SIRI,SPY,TSLA
0,0,0.021331,0.034177,1.195253e-01,0.119340,-7.441924e-20,-1.382105e-20,7.611352e-01,-2.703717e-20
1,1,0.021718,0.034195,1.149401e-01,0.127693,-7.433554e-20,-1.380818e-20,7.573668e-01,-2.700572e-20
2,2,0.022105,0.034246,1.103549e-01,0.136047,-7.425555e-20,-1.378943e-20,7.535985e-01,-2.697785e-20
3,3,0.022492,0.034332,1.057696e-01,0.144400,-7.417209e-20,-1.377437e-20,7.498301e-01,-2.694739e-20
4,4,0.022879,0.034451,1.011844e-01,0.152754,-7.409080e-20,-1.376028e-20,7.460618e-01,-2.691737e-20
...,...,...,...,...,...,...,...,...,...
96,96,0.058478,0.109167,-1.595648e-23,0.958209,-8.920475e-24,-1.988706e-23,-1.165450e-23,4.179051e-02
97,97,0.058864,0.110662,-1.307213e-23,0.968657,-4.685686e-24,-2.811167e-23,-6.292923e-24,3.134288e-02
98,98,0.059251,0.112183,-5.470587e-24,0.979105,6.131261e-25,-2.587199e-24,5.522640e-25,2.089526e-02
99,99,0.059638,0.113730,-1.971511e-23,0.989552,-1.078755e-23,-3.546617e-23,-1.707753e-23,1.044763e-02


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [56]:
# YOUR CHANGES HERE

df.to_csv("efficient_frontier.tsv", sep="\t", index=False)


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [57]:
# YOUR CHANGES HERE

from collections import Counter

expected_returns = pd.read_csv("estimated_returns.tsv", sep="\t")
cov_matrix = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

mu = expected_returns['estimated_return'].values
assets = expected_returns['asset'].values
Sigma = cov_matrix.loc[assets, assets].values  

n_simulations = 1000
n_samples = 23  

max_return_assets = []

for _ in range(n_simulations):
    simulated_returns = np.random.multivariate_normal(mu, Sigma, size=n_samples)

    estimated_mu_sim = simulated_returns.mean(axis=0)

    best_asset_idx = np.argmax(estimated_mu_sim)
    best_asset = assets[best_asset_idx]
    max_return_assets.append(best_asset)

asset_counts = Counter(max_return_assets)

stability_df = pd.DataFrame({
    'asset': list(asset_counts.keys()),
    'times_best': list(asset_counts.values())
})

stability_df['percentage'] = 100 * stability_df['times_best'] / n_simulations

stability_df = stability_df.sort_values(by='times_best', ascending=False).reset_index(drop=True)

stability_df


,asset,times_best,percentage
0,AU,737,73.7
1,TSLA,183,18.3
2,SIMO,73,7.3
3,AAPL,3,0.3
4,SIRI,2,0.2
5,SPY,2,0.2


In [58]:
#SPY is less than 1%
probability_df = pd.DataFrame({
    'asset': list(asset_counts.keys()),
    'probability': [count / n_simulations for count in asset_counts.values()]
})
probability_df

,asset,probability
0,TSLA,0.183
1,AU,0.737
2,SIMO,0.073
3,SIRI,0.002
4,SPY,0.002
5,AAPL,0.003


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [59]:
# YOUR CHANGES HERE

probability_df.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

In [60]:
acknowledgements = [
['1. https://blog.quantinsti.com/portfolio-optimization-methods/', 
 'To learn more about Python and how CVXPY and portfolio optimization works'],
['2. https://github.com/tthustla/efficient_frontier/blob/master/Efficient%20_Frontier_implementation.ipynb',
 'To learn more about how Modern Portfolio Theory, what it means and how it is implemented into python'],
['3. https://github.com/cvxgrp/cvxportfolio/tree/master/cvxportfolio', 'To look at specific examples of the cvxpy and how risks, returns and specific stocks were calculated for more information']
]


columns = ['Source', 'Used For/Reason']

ack_df = pd.DataFrame(acknowledgements, columns=columns)

ack_df




,Source,Used For/Reason
0,1. https://blog.quantinsti.com/portfolio-optim...,To learn more about Python and how CVXPY and p...
1,2. https://github.com/tthustla/efficient_front...,To learn more about how Modern Portfolio Theor...
2,3. https://github.com/cvxgrp/cvxportfolio/tree...,To look at specific examples of the cvxpy and ...


In [61]:
# YOUR CHANGES HERE

ack_df.to_csv("acknowledgements.txt", sep="\t", index=False)

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.